In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

import numpy as np

from nltk.corpus import movie_reviews, stopwords

import warnings
warnings.filterwarnings("ignore")

# Настройка параметров
В этом задании вам предстоит поэкспериментировать с параметрами вашей модели для сентимент-анализа. Все задания выполняются на том же датасете, что и на прошлой неделе.

In [2]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]
reviews = negfeats + posfeats

neg_classes = [0] * len(negids)
pos_classes = [1] * len(posids)
classes = neg_classes + pos_classes

In [3]:
def print_to_file(value, name):
    print(name, value)
    with open(name, "w") as file:
        file.write(str(value))

1. Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: 

    а) pipeline из CountVectorizer() и LogisticRegression(), 

    б) pipeline из TfidfVectorizer() и LogisticRegression(). 

В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

In [4]:
def get_pipe_scores(vectorizer, clf):
    count_pipe = Pipeline([('vectorizer', vectorizer), ('clf', clf)])
    return cross_val_score(count_pipe, reviews, classes, cv = 5)
def get_str_list(lst):
    return ' '.join(list(map(str, lst)))

In [5]:
count_scores = get_pipe_scores(CountVectorizer(), LogisticRegression(solver='liblinear'))
tfidf_scores = get_pipe_scores(TfidfVectorizer(), LogisticRegression(solver='liblinear'))
means_stds = get_str_list([count_scores.mean(), count_scores.std(), tfidf_scores.mean(), tfidf_scores.std()])
print_to_file(means_stds, '1.txt')

1.txt 0.841 0.01677796173556255 0.8210000000000001 0.004062019202317978


2. Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

In [6]:
min_df_10_scores = get_pipe_scores(CountVectorizer(min_df=10), LogisticRegression(solver='liblinear'))
min_df_50_scores = get_pipe_scores(TfidfVectorizer(min_df=50), LogisticRegression(solver='liblinear'))
min_df_means = get_str_list([min_df_10_scores.mean(), min_df_50_scores.mean()])
print_to_file(min_df_means, '2.txt')

2.txt 0.8390000000000001 0.8155000000000001


3. Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

In [13]:
clfs = [LogisticRegression(solver='liblinear'), LinearSVC(), SGDClassifier(random_state=42)] 
# SGDClassifier каждый раз будет давать разные результаты, поэтому лучше зафиксировать random_state для грейдера 
means = []
for clf in clfs:
    mean = get_pipe_scores(CountVectorizer(), clf).mean()
    means.append(mean)
print_to_file(min(means), '3.txt')

3.txt 0.74


4. Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

In [8]:
nltk_stop_scores = get_pipe_scores(CountVectorizer(stop_words = stopwords.words('english')), 
                                   LogisticRegression(solver='liblinear'))
sklrn_stop_scores = get_pipe_scores(CountVectorizer(stop_words = 'english'), 
                                   LogisticRegression(solver='liblinear'))
stop_means = get_str_list([nltk_stop_scores.mean(), sklrn_stop_scores.mean()])
print_to_file(stop_means, '4.txt')

4.txt 0.8414999999999999 0.8390000000000001


5. Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [11]:
vectorizers = [CountVectorizer(ngram_range = [1,2]),
               CountVectorizer(ngram_range = [3,5], analyzer='char_wb')]
means = []
for vectorizer in vectorizers:
    mean = get_pipe_scores(vectorizer, LogisticRegression(solver='liblinear')).mean()
    means.append(mean)
ngram_means = get_str_list(means)
print_to_file(ngram_means, '5.txt')

5.txt 0.8525 0.82
